In [5]:
import os
import cv2
import glob
import random
import warnings
import numpy as np
from PIL import Image
import disarray as da
import seaborn as sns
import tensorflow as tf
from imutils import paths
from tensorflow.keras.utils import load_img, img_to_array
from itertools import cycle
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input

In [6]:
import os
import glob
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

## ASM

In [7]:
base_path = '/home/sanjeev/ML_Dataset/microsoft_dataset/asm_img/'

In [8]:
cur_dir = os.getcwd()
os.chdir(base_path)

In [9]:
cur_dir

'/home/sanjeev/ML_Dataset/microsoft_dataset/asm_img'

In [10]:
img_list = glob.glob(os.path.join(base_path, '*/*.png'))

In [11]:
categories = os.listdir(base_path)
labels = [i for i in range(len(categories))]
label_dict = dict(zip(categories, labels))

In [12]:
cur_dir = os.getcwd()
os.chdir(base_path)  # the parent folder with sub-folders

# Get number of samples per family
list_fams = sorted(os.listdir(os.getcwd()), key=str.lower)  # vector of strings with family names
no_imgs = []  # No. of samples per family

for i in range(len(list_fams)):
    os.chdir(list_fams[i])
    len1 = len(glob.glob('*.png'))  # assuming the images are stored as 'png'
    no_imgs.append(len1)
    os.chdir('..')
num_samples = np.sum(no_imgs)  # total number of all samples

# Compute the labels
y = np.zeros(num_samples)
pos = 0
label = 0
fam = []
for i in no_imgs:
    print ("Label:%2d\tFamily: %15s\tNumber of images: %d" % (label, list_fams[label], i))
    for j in range(i):
        y[pos] = label
        pos += 1
        fam.append(list_fams[label])
    label += 1
num_classes = label

Label: 0	Family: .ipynb_checkpoints	Number of images: 0
Label: 1	Family:           Gatak	Number of images: 1013
Label: 2	Family:    Kelihos_ver1	Number of images: 398
Label: 3	Family:    Kelihos_ver3	Number of images: 2942
Label: 4	Family:        Lollipop	Number of images: 2478
Label: 5	Family:  Obfuscator.ACY	Number of images: 1228
Label: 6	Family:          Ramnit	Number of images: 1541
Label: 7	Family:           Simda	Number of images: 42
Label: 8	Family:          Tracur	Number of images: 751
Label: 9	Family:           Vundo	Number of images: 475


In [30]:
asm_features = np.load('/home/sanjeev/DL_Exp_Malware/DL_Prasad_Exp/Microsoft_PoC/Experiment_2/asm_plots/MobileNetV3_Large_X.npy')
y = np.load('/home/sanjeev/DL_Exp_Malware/DL_Prasad_Exp/Microsoft_PoC/Experiment_2/asm_plots/MobileNetV3_Large_y.npy')

In [31]:
x.shape

(10868, 1024)

In [32]:
bytes_fetaures=np.load('/home/sanjeev/DL_Exp_Malware/DL_Prasad_Exp/Microsoft_PoC/Experiment_2/bytes_plots/MobileNetV3_Large_X.npy')

In [33]:
X_asm_train, X_asm_test, y_train, y_test = train_test_split(asm_features, y, test_size=0.2, random_state=42)
X_byte_train, X_byte_test, _, _ = train_test_split(bytes_fetaures, y, test_size=0.2, random_state=42)

## mlp

In [35]:
import pandas as pd
import disarray as da
from sklearn import tree
from sklearn import metrics
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix, matthews_corrcoef, roc_curve, auc

In [40]:
%%time
mlp_asm = MLPClassifier(hidden_layer_sizes=(256,128), max_iter=300, random_state=42)
mlp_asm.fit(X_asm_train, y_train)

CPU times: user 43min 44s, sys: 44min 55s, total: 1h 28min 40s
Wall time: 1min 23s


MLPClassifier(hidden_layer_sizes=(256, 128), max_iter=300, random_state=42)

In [37]:
# hidden_layer_sizes=(256,128)
# → Your network has 2 hidden layers:

# First hidden layer: 256 neurons

# Second hidden layer: 128 neurons

# If you write hidden_layer_sizes=(100,)
# → One hidden layer with 100 neurons

# If you write hidden_layer_sizes=(512,256,128)
# → Three hidden layers: 512 → 256 → 128

# If you leave it default (hidden_layer_sizes=(100,))
# → One hidden layer with 100 neurons

In [39]:
%%time
mlp_byte = MLPClassifier(hidden_layer_sizes=(256,128), max_iter=300, random_state=42)
mlp_byte.fit(X_byte_train, y_train)

CPU times: user 48min 16s, sys: 49min 16s, total: 1h 37min 32s
Wall time: 1min 35s


MLPClassifier(hidden_layer_sizes=(256, 128), max_iter=300, random_state=42)

In [54]:
p_asm_train = mlp_asm.predict_proba(X_asm_train)

In [56]:
p_asm_test  = mlp_asm.predict_proba(X_asm_test)


In [58]:
p_byte_train = mlp_byte.predict_proba(X_byte_train)
p_byte_test  = mlp_byte.predict_proba(X_byte_test)


In [59]:
fused_train = np.concatenate([p_asm_train, p_byte_train], axis=1)

In [60]:
fused_test  = np.concatenate([p_asm_test, p_byte_test], axis=1)


In [61]:
fusion_mlp = MLPClassifier(hidden_layer_sizes=(128,64), max_iter=300, random_state=42)

In [62]:
results_fusion = evaluate_model(fusion_mlp, fused_train, y_train, fused_test, y_test, "Fusion MLP")


=== Fusion MLP ===
                precision    recall  f1-score   support

         Gatak       0.95      0.98      0.96       216
  Kelihos_ver1       0.99      0.97      0.98        78
  Kelihos_ver3       1.00      1.00      1.00       607
      Lollipop       0.95      0.97      0.96       485
Obfuscator.ACY       0.92      0.92      0.92       237
        Ramnit       0.97      0.95      0.96       303
         Simda       0.60      0.43      0.50         7
        Tracur       0.96      0.94      0.95       145
         Vundo       0.97      0.94      0.95        96

      accuracy                           0.97      2174
     macro avg       0.92      0.90      0.91      2174
  weighted avg       0.96      0.97      0.96      2174



In [64]:
results_asm = evaluate_model(mlp_asm, X_asm_train, y_train, X_asm_test, y_test, "MLP ASM")
results_byte = evaluate_model(mlp_byte, X_byte_train, y_train, X_byte_test, y_test, "MLP Byte")


=== MLP ASM ===
                precision    recall  f1-score   support

         Gatak       0.94      0.82      0.88       216
  Kelihos_ver1       0.97      0.95      0.96        78
  Kelihos_ver3       0.95      0.98      0.96       607
      Lollipop       0.92      0.91      0.91       485
Obfuscator.ACY       0.94      0.87      0.91       237
        Ramnit       0.95      0.96      0.96       303
         Simda       0.43      0.43      0.43         7
        Tracur       0.77      0.92      0.84       145
         Vundo       0.70      0.74      0.72        96

      accuracy                           0.91      2174
     macro avg       0.84      0.84      0.84      2174
  weighted avg       0.92      0.91      0.92      2174


=== MLP Byte ===
                precision    recall  f1-score   support

         Gatak       0.96      0.95      0.95       216
  Kelihos_ver1       0.99      0.95      0.97        78
  Kelihos_ver3       1.00      0.99      1.00       607
      Lol

In [65]:
import pandas as pd
df_results = pd.DataFrame([results_asm, results_byte, results_fusion])
print("\nFinal Comparison:\n")
print(df_results)


Final Comparison:

        Model  Accuracy  Precision    Recall  F1-score  Train Time (s)  \
0     MLP ASM  0.914903   0.917625  0.914903  0.915259       83.808307   
1    MLP Byte  0.932843   0.934450  0.932843  0.933132       92.991858   
2  Fusion MLP  0.965041   0.964868  0.965041  0.964837        6.042202   

   Prediction Time (s)  
0             0.016952  
1             0.016790  
2             0.010304  
